In [10]:
!pip install imbalanced-learn rdkit tqdm
import imblearn

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import rdkit
import seaborn as sn
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn import tree, model_selection
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import sklearn
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict

In [13]:
df = pd.read_csv("Final_data_with_labels.csv")
df.head()

,Ligand SMILES,Ligand InChI,Target Name,Ki (nM),IC50 (nM),Kd (nM),EC50 (nM),Activity
0,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,InChI=1S/C19H24IN3O8/c20-12-6-4-11(5-7-12)16(2...,Glutamate carboxypeptidase 2,127.272727,NaN,NaN,NaN,1
1,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,InChI=1S/C19H24IN3O8/c20-12-6-4-11(5-7-12)16(2...,Glutamate carboxypeptidase 2,127.272727,NaN,NaN,NaN,1
2,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,InChI=1S/C60H59BrN8O14/c61-39-8-4-36(5-9-39)35...,Glutamate carboxypeptidase 2,127.272727,NaN,NaN,NaN,1
3,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,InChI=1S/C60H59BrN8O14/c61-39-8-4-36(5-9-39)35...,Glutamate carboxypeptidase 2,127.272727,NaN,NaN,NaN,1
4,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,InChI=1S/C19H24IN3O8/c20-12-6-4-11(5-7-12)16(2...,Glutamate carboxypeptidase 2,127.272727,NaN,NaN,NaN,1


In [14]:
columns_to_drop = [ "Ligand InChI", "Target Name", "Ki (nM)", "IC50 (nM)", "Kd (nM)", "EC50 (nM)"]  # List of columns to remove
data = df.drop(columns=columns_to_drop)
data.head()

,Ligand SMILES,Activity
0,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1
1,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1
2,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,1
3,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,1
4,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1


In [15]:
#include 2d structure of the molecules
PandasTools.AddMoleculeColumnToFrame(data,'Ligand SMILES','Structure')
data.head()

[09:19:59] non-ring atom 14 marked aromatic
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 29 30 32 33 34
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 23 24 25 26 27 28 30 31 32
[09:20:00] Explicit valence for atom # 21 N, 4, is greater than permitted
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] non-ring atom 25 marked aromatic
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:00] Can't kekulize mol.  Unkekulized atoms: 25 26 27 2

,Ligand SMILES,Activity,Structure
0,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1,<rdkit.Chem.rdchem.Mol object at 0x7846ed37c7b0>
1,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1,<rdkit.Chem.rdchem.Mol object at 0x7846ed37dd20>
2,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,1,<rdkit.Chem.rdchem.Mol object at 0x7846ed37d070>
3,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...,1,<rdkit.Chem.rdchem.Mol object at 0x7846ed37d8c0>
4,OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...,1,<rdkit.Chem.rdchem.Mol object at 0x7846ed37c9e0>


In [16]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np

def filter_valid_smiles(data, smiles_column='Ligand SMILES'):
    """
    Removes rows with invalid SMILES and retains corresponding class labels.

    Parameters:
    - data (pd.DataFrame): Dataset containing SMILES and class labels.
    - smiles_column (str): Column name containing SMILES strings.

    Returns:
    - pd.DataFrame: Filtered dataset with valid SMILES and corresponding labels.
    """
    valid_rows = []

    for index, row in data.iterrows():
        smiles = row[smiles_column]
        mol = Chem.MolFromSmiles(smiles) if isinstance(smiles, str) else None

        if mol is not None:
            row['Mol'] = mol  # Add the valid RDKit Mol object
            valid_rows.append(row)  # Keep valid entries

    filtered_data = pd.DataFrame(valid_rows)

    print(f"Removed {len(data) - len(filtered_data)} invalid SMILES.")
    return filtered_data

def generate_morgan_fps(data, mol_column='Mol', radius=2, n_bits=2048):
    """
    Generates Morgan fingerprints for valid molecules.

    Parameters:
    - data (pd.DataFrame): Filtered dataset with valid molecules.
    - mol_column (str): Column name containing RDKit Mol objects.
    - radius (int): Radius for Morgan fingerprints.
    - n_bits (int): Bit size for fingerprint.

    Returns:
    - np.array: Array of Morgan fingerprints.
    - pd.Series: Corresponding class labels.
    """
    fingerprints = []
    labels = data['Activity'].values  # Preserve class labels

    for mol in data[mol_column]:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        fingerprints.append(fp)

    return np.array(fingerprints, dtype=object), labels


In [17]:
df = pd.read_csv("Final_data_with_labels.csv")

# Ensure correct column names
df.columns = df.columns.str.strip()

# Remove invalid SMILES (keeps only valid ones along with class labels)
filtered_df = filter_valid_smiles(df, smiles_column='Ligand SMILES')

# Generate fingerprints and retain corresponding class labels
fingerprints, class_labels = generate_morgan_fps(filtered_df)

# Convert fingerprints to a usable format (e.g., NumPy array for ML)
fingerprint_array = np.array([list(fp) for fp in fingerprints])


# Verify shape
print(f"Final dataset shape: {fingerprint_array.shape},  shape: {class_labels.shape}")


[09:20:24] non-ring atom 14 marked aromatic
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 29 30 32 33 34
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 23 24 25 26 27 28 30 31 32
[09:20:29] Explicit valence for atom # 21 N, 4, is greater than permitted
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] non-ring atom 25 marked aromatic
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:29] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:20:30] Can't kekulize mol.  Unkekulized atoms: 25 26 27 2

Removed 109 invalid SMILES.


Streaming output truncated to the last 5000 lines.
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:21:15] DEPRECATION WARNING: please use MorganGenerator
[09:2

Final dataset shape: (35097, 2048),  shape: (35097,)


In [18]:
def compute_morgan_fingerprints(data, smiles_column='Ligand SMILES', radius=2, n_bits=2048):
    """
    Compute Morgan fingerprints from a DataFrame containing ligand SMILES.
    Invalid SMILES are skipped.

    Parameters:
    - df (pd.DataFrame): DataFrame containing SMILES strings.
    - smiles_column (str): Column name containing SMILES.
    - radius (int): Radius for the Morgan fingerprint.
    - n_bits (int): Number of bits for the fingerprint.

    Returns:
    - np.array: Numpy array of Morgan fingerprints.
    - pd.DataFrame: Filtered DataFrame containing only valid SMILES.
    """

    valid_mols = []
    fingerprints = []

    for smiles in tqdm(df[smiles_column], desc="Processing SMILES"):
        mol = Chem.MolFromSmiles(smiles)

        if mol is None:
            print(f"Skipping invalid SMILES: {smiles}")
            continue  # Skip invalid SMILES

        # Compute Morgan fingerprint
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)

        valid_mols.append(smiles)  # Keep only valid SMILES
        fingerprints.append(fp)  # Store fingerprint

    # Create a new DataFrame with only valid SMILES
    filtered_df = pd.DataFrame({smiles_column: valid_mols})

    return np.array(fingerprints), filtered_df



In [19]:
# function calling
fingerprints, valid_df = compute_morgan_fingerprints(filtered_df, smiles_column='Ligand SMILES')

print("\nGenerated Fingerprints:")
print(fingerprints)

print("\nFiltered DataFrame with Valid SMILES:")
print(valid_df)

Processing SMILES:   0%|          | 0/35206 [00:00<?, ?it/s][09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenerator
[09:22:11] DEPRECATION WARNING: please use MorganGenera

Skipping invalid SMILES: CC(O)(CS(=O)(=O)c1ccc(F)cc1)[c](=O):c:n-c1ccc(C#N)c(c1)C(F)(F)F


[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerator
[09:22:13] DEPRECATION WARNING: please use MorganGenerat

Skipping invalid SMILES: Cc1nc2ccc(cc2n1)-n1ncc(C(=O)c2cc3ccc(OC4CCOCC4)cc3[nH]2)c1N
Skipping invalid SMILES: C[C@H]1COCCN1Cc1ccc2cc([nH]c2c1)C(=O)c1cnn(c1N)-c1ccc2nc(C)nc2c1
Skipping invalid SMILES: Cc1nc2ccc(cc2n1)-n1ncc(C(=O)c2cc3cc(CN4CCCC4)ccc3[nH]2)c1N
Skipping invalid SMILES: CN1CCN(Cc2ccc3[nH]c(cc3c2)C(=O)c2cnn(c2N)-c2ccc3nc(C)nc3c2)CC1
Skipping invalid SMILES: COc1cc(OC)cc(c1)C(\O)=C1\CC=c2ncc(cc2=[N]1CCO)-c1cnn(C)c1
Skipping invalid SMILES: Cc1nc2ccc(cc2n1)-n1ncc(C(=O)c2ccc3cc[nH]c3c2)c1N


[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerator
[09:22:16] DEPRECATION WARNING: please use MorganGenerat

...

Skipping invalid SMILES: C[N]1=CN=C2C=C(Oc3ccc(Nc4ncnc5cnc(nc45)N4CC[C@@H](CC#N)C4)cc3Cl)C=CC12


[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use MorganGenerator
Processing SMILES:  85%|████████▍ | 29764/35206 [00:49<00:11, 484.24it/s][09:23:00] DEPRECATION WARNING: please use MorganGenerator
[09:23:00] DEPRECATION WARNING: please use

Skipping invalid SMILES: COc1cc2c(Nc3ccc(Cl)c(Cl)c3)ncnc2cc1OCC1=N[N](CS(=O)(=O)c2ccccn2)=CO1
Skipping invalid SMILES: COC1=CC=[N](C=C1)c1nc(COc2cc3ncnc(Nc4ccc(Cl)c(Cl)c4)c3cc2OC)no1


[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerator
[09:23:03] DEPRECATION WARNING: please use MorganGenerat


Generated Fingerprints:
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Filtered DataFrame with Valid SMILES:
                                           Ligand SMILES
0      OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...
1      OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...
2      OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...
3      OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...
4      OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...
...                                                  ...
35092           CC1Nc2ccc(cc2C2(CCCCC2)O1)-c1sc(cc1C)C#N
35093  CC(=O)[C@H]1CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(...
35094               CC1Nc2ccc(cc2C(C)(C)O1)-c1csc(c1)C#N
35095             CC1(C)OC(=O)Nc2ccc(cc12)-c1[nH]ccc1C#N
35096             Cn1c(ccc1-c1ccc2NC(=O)OC(C)(C)c2c1)C#N

[35097 rows x 1 columns]


In [20]:
# Convert fingerprints to DataFrame
df_fingerprints = pd.DataFrame(fingerprints)
Labels = pd.DataFrame(class_labels)

In [21]:
df_fingerprints.head()
Labels.head()

,0
0,1
1,1
2,1
3,1
4,1


In [22]:
# Append column Activity in fingerprints
df_fingerprints['Activity'] = Labels
df_fingerprints.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,Activity
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
# Count occurrences of each class
print("\nClass Distribution Before Balancing:")
class_counts = df['Activity'].value_counts()
print(class_counts)



Class Distribution Before Balancing:
Activity
1    28478
0     6728
Name: count, dtype: int64


In [ ]:
df_fingerprints.to_csv('fingerprints_actual.csv', index=False)

class imbalance handling

In [24]:
# Function for SMILES augmentation
def augment_smiles(smiles):
    """Generate a randomized version of a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol, doRandom=True) if mol else smiles

In [25]:
# Identify minority class
minority_class = df_fingerprints['Activity'].value_counts().idxmin()
df_minority = df_fingerprints[df_fingerprints['Activity'] == minority_class].copy()

In [26]:
# Calculate how many samples to add for balance
num_augmentations = df_fingerprints['Activity'].value_counts().max() - len(df_minority)


In [27]:
# Apply augmentation to the minority class
print(data.head())
column_name = 'Ligand SMILES'
augmented_smiles = [augment_smiles(smiles) for smiles in df[df['Activity'] == minority_class][column_name].sample(n=num_augmentations, replace=True)]


                                       Ligand SMILES  Activity  \
0  OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...         1   
1  OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...         1   
2  OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...         1   
3  OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCN(Cc1ccc(Br)cc...         1   
4  OC(=O)CC[C@H](NC(=O)N[C@@H](CCCCNC(=O)c1ccc(I)...         1   

                                          Structure  
0  <rdkit.Chem.rdchem.Mol object at 0x7846ed37c7b0>  
1  <rdkit.Chem.rdchem.Mol object at 0x7846ed37dd20>  
2  <rdkit.Chem.rdchem.Mol object at 0x7846ed37d070>  
3  <rdkit.Chem.rdchem.Mol object at 0x7846ed37d8c0>  
4  <rdkit.Chem.rdchem.Mol object at 0x7846ed37c9e0>  


[09:25:31] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:25:34] non-ring atom 25 marked aromatic
[09:25:37] non-ring atom 25 marked aromatic
[09:25:38] non-ring atom 25 marked aromatic
[09:25:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9
[09:25:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 8 9


In [28]:
# Create new DataFrame with augmented SMILES
df_augmented = df_minority.sample(n=num_augmentations, replace=True).copy()
df_augmented['Ligand SMILES'] = augmented_smiles  # Replace SMILES with augmented ones


In [29]:
# Merge original and augmented data
df_balanced = pd.concat([df_fingerprints, df_augmented]).reset_index(drop=True)
df_balanced.head()

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,Activity,Ligand SMILES
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,NaN
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,NaN
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,NaN


In [30]:
df_balanced.tail()

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,Activity,Ligand SMILES
56737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1ccc(c(Nc2ccc(Oc3ccc(cc3)C(F)(F)F)cc2)c1)C(=O)O
56738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1(-c2nc(sc2)N2CCOCC2)ccccn1
56739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,c1c2ccccc2ccc1-c1cc(OC)c(c(c1)C=O)O
56740,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,[C@H](C(N[C@H](C(=O)N[C@@H](CO)C(NCC(N[C@@H](C...
56741,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C1(=C2\C(=O)NC(N2)=N)\c2c(c(Br)[nH]c2C(=O)NCC1)Br


In [31]:
df_balanced = df_balanced.drop(columns='Ligand SMILES')
df_balanced.head()

,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,Activity
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
# Check class distribution after balancing
print("\nClass Distribution After Balancing:")
print(df_balanced['Activity'].value_counts())


Class Distribution After Balancing:
Activity
1    28371
0    28371
Name: count, dtype: int64


In [33]:
# Save the balanced dataset
df_balanced.to_csv('balanced_smiles_data_1.csv', index=False)